In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
import time
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from inspect import getfullargspec,signature

import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import Dataset
from torchvision import datasets

import torch as ch
from robustness.datasets import CINIC
ds = CINIC('/tmp/cinic')

from sklearn import metrics

%matplotlib inline
%config InlineBackend.figure_format = 'svg'


In [2]:
############################
# Use pytorch pretrained Resnet18
#######################

In [16]:
        
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")
    
# load STD model 
############################
class CinicResNet(nn.Module):
  def __init__(self, in_channels=3):
    super(CinicResNet, self).__init__()

    # Load resnet model from torchvision.models in Pytorch
    self.model = models.resnet50(pretrained=False)

    # Typical image conv first layer
    self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
    # Change the output layer to output 10 classes instead of 1000 classes
    num_ftrs = self.model.fc.in_features
    self.model.fc = nn.Linear(num_ftrs, 10)

  def forward(self, x):
    return self.model(x)

# load STD model 
model_std = CinicResNet().to(device)

# params you need to specify:
epochs = 120
batch_size = 128
print(model_std)
model_state_dict = ch.load("./CinicSTD50Net")
model_std.load_state_dict(model_state_dict)
model_std.to("cpu")

CinicResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

CinicResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

In [17]:
### Initiate ADV model 
from robustness.model_utils import make_and_restore_model
model_adv, _ = make_and_restore_model(arch='resnet50', dataset=ds,
              resume_path= '/home/ashraf.haddad/mlpr/CINIC-50Store-ADV/checkpoint.pt.best') #'/tmp/35daedae-1b39-4941-ad08-8bd6459c1bd8/checkpoint.pt.best')
            
model_adv.eval()
pass

=> loading checkpoint '/home/ashraf.haddad/mlpr/CINIC-50Store-ADV/checkpoint.pt.best'
=> loaded checkpoint '/home/ashraf.haddad/mlpr/CINIC-50Store-ADV/checkpoint.pt.best' (epoch 56)


In [18]:
#only load the validation set
ATTACK_EPS = 0.5
ATTACK_STEPSIZE = 0.1
ATTACK_STEPS = 10
NUM_WORKERS = 8
BIG_BATCH_SIZE = 128 #128

_, test_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BIG_BATCH_SIZE)
test_enum = enumerate(test_loader)

==> Preparing dataset cinic..


In [19]:
kwargs = {
    'constraint':'2', # use L2-PGD
    'eps': ATTACK_EPS, # L2 radius around original image
    'step_size': ATTACK_STEPSIZE,
    'iterations': ATTACK_STEPS,
    'do_tqdm': True,
}


In [20]:
print(torch.cuda.is_available())
torch.cuda.empty_cache() 

True


In [21]:
# measure accuracy on the validation set but in batch sizes,
# since robust model can't memory-handle large image sizes at once
acc_list = []
from sklearn import metrics

for _ in range(len(test_loader)):
    _, (im, label) = next(test_enum)
    im,label = im.to(device),label.to(device)
    
    #the model can generate adv by applying max l-2 loss
    _, im_adv = model_adv(im, label, make_adv=True, **kwargs)
    im_adv_transformed = transforms.Resize((224, 224)).forward(im_adv)
    
    im_adv = im_adv.to(device='cpu')
    im_adv_transformed = im_adv_transformed.to(device='cpu')
    
    # Get predicted labels for adversarial examples using 
    pred = model_std(im_adv_transformed)
    label_pred = ch.argmax(pred, dim=1)
    
    im,label = im.to(device='cpu'),label.to(device='cpu')
    im_adv = im_adv.to(device='cpu')
    im_adv_transformed = im_adv_transformed.to(device='cpu')

    #Get accuracy
    acc_list.append(metrics.accuracy_score(label ,label_pred.cpu()))
    print("Accuracy:",metrics.accuracy_score(label ,label_pred.cpu()))

Current loss: 1.3220775127410889: 100%|█████████| 10/10 [00:02<00:00,  4.83it/s]


Accuracy: 0.171875


Current loss: 1.3351069688796997: 100%|█████████| 10/10 [00:02<00:00,  4.79it/s]


Accuracy: 0.171875


Current loss: 1.323685646057129: 100%|██████████| 10/10 [00:02<00:00,  4.89it/s]


...

Accuracy: 0.203125


Current loss: 1.3724976778030396: 100%|█████████| 10/10 [00:04<00:00,  2.25it/s]


Accuracy: 0.1640625


Current loss: 1.1869122982025146: 100%|█████████| 10/10 [00:00<00:00, 12.05it/s]


Accuracy: 0.125


In [23]:

# predict and measure the accuracy 

## Generate adv examples 

print("Average over all batches ",   sum(acc_list) / len(acc_list))   


Average over all batches  0.15269886363636365
